<a href="https://colab.research.google.com/github/sandeepgundeboina/LearningSpark/blob/main/SparkBadRecords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pyspark

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark=SparkSession.builder.appName('BadRecords').getOrCreate()

In [2]:
data=[
    ('JAN',234,2000,30),
    ('FEB',343,1788,20),
    ('MAR',454,2897,30),
    ('APR',344,2000,30),
    ('MAY',213,1098,30),
    ('JUN',2344,1876,30),
    ('JUL','twenty',2000,30),
    (80,987,2132,'Lest'),
    ('SEP',334,2000,30),
    ('OCT',212,'ewer',30)
]



In [3]:
df=spark.createDataFrame(data,['month','qty','price','discount'])
df.show()

+-----+------+-----+--------+
|month|   qty|price|discount|
+-----+------+-----+--------+
|  JAN|   234| 2000|      30|
|  FEB|   343| 1788|      20|
|  MAR|   454| 2897|      30|
|  APR|   344| 2000|      30|
|  MAY|   213| 1098|      30|
|  JUN|  2344| 1876|      30|
|  JUL|twenty| 2000|      30|
|   80|   987| 2132|    Lest|
|  SEP|   334| 2000|      30|
|  OCT|   212| ewer|      30|
+-----+------+-----+--------+



In [4]:
df.write.csv('/content/drive/MyDrive/Abc/postgreSQL/examples/bad_records3.csv', header=True)

In [5]:
data=spark.read.csv('/content/drive/MyDrive/Abc/postgreSQL/examples/bad_records1.csv', header=True)
data.show()

+-----+------+-----+--------+
|month|   qty|price|discount|
+-----+------+-----+--------+
|  JUN|  2344| 1876|      30|
|  JUL|twenty| 2000|      30|
|   80|   987| 2132|    Lest|
|  SEP|   334| 2000|      30|
|  OCT|   212| ewer|      30|
|  JAN|   234| 2000|      30|
|  FEB|   343| 1788|      20|
|  MAR|   454| 2897|      30|
|  APR|   344| 2000|      30|
|  MAY|   213| 1098|      30|
+-----+------+-----+--------+



In [6]:
schema1=StructType([
    StructField('month',StringType(),True),
    StructField('qty',IntegerType(),True),
    StructField('price',IntegerType(),True),
    StructField('discount',IntegerType(),True)
])

####**Permissive**

    for read a file by default the mode is permissive, so it will continue with file reading despite of bad records.
    Bad records are replaced with Null values.

In [7]:
data=spark.read.csv('/content/drive/MyDrive/Abc/postgreSQL/examples/bad_records1.csv', header=True, schema=schema1)
data.show()

+-----+----+-----+--------+
|month| qty|price|discount|
+-----+----+-----+--------+
|  JUN|2344| 1876|      30|
|  JUL|NULL| 2000|      30|
|   80| 987| 2132|    NULL|
|  SEP| 334| 2000|      30|
|  OCT| 212| NULL|      30|
|  JAN| 234| 2000|      30|
|  FEB| 343| 1788|      20|
|  MAR| 454| 2897|      30|
|  APR| 344| 2000|      30|
|  MAY| 213| 1098|      30|
+-----+----+-----+--------+



In [8]:
data4=spark.read.format('csv').option('header','True').option('mode','PERMISSIVE').schema(schema1).load('/content/drive/MyDrive/Abc/postgreSQL/examples/bad_records1.csv')
data4.show()

+-----+----+-----+--------+
|month| qty|price|discount|
+-----+----+-----+--------+
|  JUN|2344| 1876|      30|
|  JUL|NULL| 2000|      30|
|   80| 987| 2132|    NULL|
|  SEP| 334| 2000|      30|
|  OCT| 212| NULL|      30|
|  JAN| 234| 2000|      30|
|  FEB| 343| 1788|      20|
|  MAR| 454| 2897|      30|
|  APR| 344| 2000|      30|
|  MAY| 213| 1098|      30|
+-----+----+-----+--------+



####**Drop Malformed**

    in DropMalformed all the bad records will be deleted and process of reading the file will continue.
    Only proper records are stored in dataframe.

In [9]:
data1=spark.read.format('csv').option('header','True').option('mode','DropMalformed').schema(schema1).load('/content/drive/MyDrive/Abc/postgreSQL/examples/bad_records1.csv')
data1.show()

+-----+----+-----+--------+
|month| qty|price|discount|
+-----+----+-----+--------+
|  JUN|2344| 1876|      30|
|  SEP| 334| 2000|      30|
|  JAN| 234| 2000|      30|
|  FEB| 343| 1788|      20|
|  MAR| 454| 2897|      30|
|  APR| 344| 2000|      30|
|  MAY| 213| 1098|      30|
+-----+----+-----+--------+



####**FailFast**

    In FailFast the process of reading the file will be stopped totally by throwing the exception

In [10]:
data2=spark.read.format('csv').option('header','True').option('mode','FailFast').schema(schema1).load('/content/drive/MyDrive/Abc/postgreSQL/examples/bad_records1.csv')
data2.show()

Py4JJavaError: An error occurred while calling o97.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 12.0 failed 1 times, most recent failure: Lost task 0.0 in stage 12.0 (TID 13) (3b013414dbac executor driver): org.apache.spark.SparkException: [MALFORMED_RECORD_IN_PARSING.WITHOUT_SUGGESTION] Malformed records are detected in record parsing: [JUL,null,2000,30].
Parse Mode: FAILFAST. To process malformed records as null result, try setting the option 'mode' as 'PERMISSIVE'. 
	at org.apache.spark.sql.errors.QueryExecutionErrors$.malformedRecordsDetectedInRecordParsingError(QueryExecutionErrors.scala:1610)
	at org.apache.spark.sql.catalyst.util.FailureSafeParser.parse(FailureSafeParser.scala:79)
	at org.apache.spark.sql.catalyst.csv.UnivocityParser$.$anonfun$parseIterator$2(UnivocityParser.scala:456)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.

    END OF CODE

